In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-crashes-severity-prediction/sample_submission.csv
/kaggle/input/car-crashes-severity-prediction/weather-sfcsv.csv
/kaggle/input/car-crashes-severity-prediction/train.csv
/kaggle/input/car-crashes-severity-prediction/test.csv
/kaggle/input/car-crashes-severity-prediction/holidays.xml


# **Train Dataset**

In [2]:
import pandas as pd 
import datetime
import numpy as np
from sklearn.preprocessing import LabelEncoder
import numpy as np; np.random.seed(0)
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as Xet

In [3]:
data_train = pd.read_csv("../input/car-crashes-severity-prediction/train.csv") 
dTrain = pd.DataFrame(data_train)
dTrain.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


In [4]:
dTrain=dTrain.drop(columns=['Bump','Roundabout'])

In [5]:
dTrain.groupby('Side').count()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Severity,timestamp
Side,,,,,,,,,,,,,
L,325,325,325,325,325,325,325,325,325,325,325,325,325
R,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082


In [6]:
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,0.044,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,0.000,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,0.000,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,0.009,False,False,True,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,0.000,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


In [7]:
dTrain['Crossing'] = dTrain['Crossing'].astype('int')
dTrain['Give_Way'] = dTrain['Give_Way'].astype('int')
dTrain['Junction'] = dTrain['Junction'].astype('int')
dTrain['No_Exit'] = dTrain['No_Exit'].astype('int')
dTrain['Railway'] = dTrain['Railway'].astype('int')
dTrain['Stop'] = dTrain['Stop'].astype('int')
dTrain['Amenity'] = dTrain['Amenity'].astype('int')
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,R,2,2019-10-09 08:47:00


In [8]:
enc = LabelEncoder()
enc.fit(dTrain['Side'])
dTrain['Side'] = enc.transform(dTrain['Side'])

In [9]:
dTrain.groupby('Side').count()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Severity,timestamp
Side,,,,,,,,,,,,,
0,325,325,325,325,325,325,325,325,325,325,325,325,325
1,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082,6082


In [10]:
dTrain.dtypes

ID                int64
Lat             float64
Lng             float64
Distance(mi)    float64
Crossing          int64
Give_Way          int64
Junction          int64
No_Exit           int64
Railway           int64
Stop              int64
Amenity           int64
Side              int64
Severity          int64
timestamp        object
dtype: object

In [11]:
#convert timestamp to dateTime
dTrain['timestamp']=pd.to_datetime(dTrain['timestamp'])

In [12]:
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00


In [13]:
#split date from time
dTrain['date'] = [d.date() for d in dTrain['timestamp']]

In [14]:
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43,2020-03-29
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09


In [15]:
dTrain.dtypes

ID                       int64
Lat                    float64
Lng                    float64
Distance(mi)           float64
Crossing                 int64
Give_Way                 int64
Junction                 int64
No_Exit                  int64
Railway                  int64
Stop                     int64
Amenity                  int64
Side                     int64
Severity                 int64
timestamp       datetime64[ns]
date                    object
dtype: object

In [16]:
#get hours from time alone but int
dTrain['Hour'] = dTrain['timestamp'].dt.hour
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date,Hour
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25,15
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05,19
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16,19
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43,2020-03-29,19
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09,8


In [17]:
#convert from int to str
dTrain['Hour']=dTrain['Hour'].apply(str)
dTrain["Hour"] = dTrain['Hour'].astype(str)+':00:00'

In [18]:
#put hour str in time formate
dTrain['Hour'] = pd.to_datetime(dTrain['Hour'],format= '%H:%M:%S' ).dt.time

In [19]:
dTrain["Hour"] = dTrain['Hour'].astype(str)

In [20]:
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date,Hour
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25,15:00:00
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05,19:00:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16,19:00:00
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43,2020-03-29,19:00:00
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09,08:00:00


In [21]:
#date and time str to concatenate later
dTrain.dtypes

ID                       int64
Lat                    float64
Lng                    float64
Distance(mi)           float64
Crossing                 int64
Give_Way                 int64
Junction                 int64
No_Exit                  int64
Railway                  int64
Stop                     int64
Amenity                  int64
Side                     int64
Severity                 int64
timestamp       datetime64[ns]
date                    object
Hour                    object
dtype: object

In [22]:
dTrain["Hour"]= dTrain["Hour"].str.split(":", n = 1, expand = True)
dTrain['Hour'] = dTrain['Hour'].astype(int)
dTrain.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date,Hour
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25,15
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05,19
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16,19
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43,2020-03-29,19
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09,8


# **Weather Dataset**

In [23]:
data_weather = pd.read_csv("../input/car-crashes-severity-prediction/weather-sfcsv.csv") 
dWeather = pd.DataFrame(data_weather)
# Preview the first 5 lines of the loaded data 
dWeather.head()

,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No


In [24]:
dWeather['Year'] = dWeather['Year'].apply(str)
dWeather['Month'] = dWeather['Month'].apply(str)
dWeather['Day'] = dWeather['Day'].apply(str)

In [25]:
dWeather["date"] = pd.to_datetime(dWeather['Year'].astype(str)+'-'+dWeather['Month']+'-'+dWeather['Day']).dt.date
dWeather.head()

,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,date
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No,2020-07-27
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-09-30
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-06-27
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-09-07
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No,2019-10-19


In [26]:
dWeather['Weather_Condition'] = dWeather['Weather_Condition'].apply(str)

In [27]:
dWeather = dWeather.drop(columns=['Year', 'Month','Day'])

In [28]:
dWeather.head()

,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,date
0,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No,2020-07-27
1,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-09-30
2,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-06-27
3,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-09-07
4,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No,2019-10-19


In [29]:
dWeather.to_csv('weather.csv')

# **Holiday Dataset**

In [30]:
cols = ["date", "description"]
rows = []
  
# Parsing the XML file
xmlparse = Xet.parse('../input/car-crashes-severity-prediction/holidays.xml')
root = xmlparse.getroot()
for i in root:
    date = i.find("date").text
    description = i.find("description").text
  
    rows.append({"date": date,
                 "description": description})
  
dholidays = pd.DataFrame(rows, columns=cols)
  
# Writing dataframe to csv
dholidays.to_csv('holidays.csv')

In [31]:
dholidays['date']=pd.to_datetime(dholidays['date']).dt.date
dholidays.head()

,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day


In [32]:
dholidays.dtypes

date           object
description    object
dtype: object

# **Merging Datasets**

> **Merge Train with Weather by date**


In [33]:
dTrain_dWeather = dTrain.merge(dWeather, left_on=["date","Hour"], right_on=["date","Hour"], how = 'left')

In [34]:
dTrain_dWeather.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,date,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,2016-03-25,15,Scattered Clouds,NaN,NaN,64.0,58.0,23.0,10.0,No
1,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,2016-03-25,15,Partly Cloudy,NaN,NaN,66.0,54.0,17.3,10.0,No
2,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,2020-05-05,19,Mostly Cloudy / Windy,57.0,0.0,57.0,83.0,22.0,10.0,No
3,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,2020-05-05,19,Mostly Cloudy,59.0,0.0,59.0,78.0,20.0,10.0,No
4,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,...,2016-09-16,19,Clear,NaN,NaN,62.1,80.0,9.2,10.0,No


> **Merge Train & Weather with Holidays by date**

In [35]:
dfinal = dTrain_dWeather.merge(dholidays, on="date", how = 'left')

In [36]:
dfinal.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,description
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,Scattered Clouds,NaN,NaN,64.0,58.0,23.0,10.0,No,NaN
1,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,Partly Cloudy,NaN,NaN,66.0,54.0,17.3,10.0,No,NaN
2,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,Mostly Cloudy / Windy,57.0,0.0,57.0,83.0,22.0,10.0,No,NaN
3,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,Mostly Cloudy,59.0,0.0,59.0,78.0,20.0,10.0,No,NaN
4,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,...,19,Clear,NaN,NaN,62.1,80.0,9.2,10.0,No,NaN


In [37]:
dfinal['Wind_Chill(F)'] = dfinal['Wind_Chill(F)'].fillna((dfinal['Wind_Chill(F)'].mean()))
dfinal['Precipitation(in)'] = dfinal['Precipitation(in)'].fillna((dfinal['Precipitation(in)'].mean()))
dfinal['Temperature(F)'] = dfinal['Temperature(F)'].fillna((dfinal['Precipitation(in)'].mean()))

dfinal.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,description
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,Scattered Clouds,60.114079,0.00728,64.0,58.0,23.0,10.0,No,NaN
1,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,Partly Cloudy,60.114079,0.00728,66.0,54.0,17.3,10.0,No,NaN
2,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,Mostly Cloudy / Windy,57.000000,0.00000,57.0,83.0,22.0,10.0,No,NaN
3,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,Mostly Cloudy,59.000000,0.00000,59.0,78.0,20.0,10.0,No,NaN
4,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,...,19,Clear,60.114079,0.00728,62.1,80.0,9.2,10.0,No,NaN


In [38]:
dfinal['description'] = dfinal['description'].fillna(0)
dfinal["description"]=dfinal["description"].apply(lambda x: 1 if x!=0 else 0)

In [39]:
enc = LabelEncoder()
enc.fit(dfinal['Weather_Condition'])
dfinal['Weather_Condition'] = enc.transform(dfinal['Weather_Condition'])

In [40]:
dfinal.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,description
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,22,60.114079,0.00728,64.0,58.0,23.0,10.0,No,0
1,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,15,17,60.114079,0.00728,66.0,54.0,17.3,10.0,No,0
2,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,15,57.000000,0.00000,57.0,83.0,22.0,10.0,No,0
3,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,19,14,59.000000,0.00000,59.0,78.0,20.0,10.0,No,0
4,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,...,19,0,60.114079,0.00728,62.1,80.0,9.2,10.0,No,0


In [41]:
dfinal['date'] = dfinal['date'].apply(str)

In [42]:
dfinal.to_csv('finalDataset.csv')

# **Data Splitting**

In [43]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(dfinal, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']

In [44]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Lat', 'Stop', 'Hour']]
X_val = X_val[['Lat', 'Stop', 'Hour']]

# **Model Training**

In [45]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

In [46]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.747072599531616


# **Submission File Generation**


In [47]:
test_df = pd.read_csv(os.path.join('../input/car-crashes-severity-prediction/test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


In [48]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp,Severity
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31,2
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00,2
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45,2
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25,2
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58,2


In [49]:
test_df[['ID', 'Severity']].to_csv('kaggle_submission.csv', index=False)